In [ ]:
#@formatter:off
%stop_session
%region ap-east-1
%iam_role arn:aws:iam::{arnNumber}:role/{IamRoleName}
%worker_type G.1X
%glue_version 4.0
%number_of_workers 3
%idle_timeout 10
%session_id_prefix develop_db_compliance
%profile default
%connections de-db-replica-admin-connection 
#@formatter:on

In [ ]:
import sys
import re
from datetime import datetime
from pytz import timezone
import boto3
import json
import functools
import time

from pyspark.context import SparkContext  # glue 기동시 필요함.
from pyspark.sql.functions import when, col, sum as spark_sum
from pyspark.sql.functions import trim, count as spark_count

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job

class DataDiffCheck:
    def __init__(self):
        # AWS Glue job 개발을 위해 필요한 변수 초기화 
        self.sc = SparkContext.getOrCreate()
        self.glueContext = GlueContext(self.sc)
        self.spark = self.glueContext.spark_session
        self.job = Job(self.glueContext)
        self.default_arguments = {
            "--JOB_NAME": "de-daily-data-sync",
            "--DB_CONNECTION": "de-db-replica-admin-connection",
            "--DATALAKE_REGION": "{region}"
        }
        self.argv = [*sys.argv,
                     *(a for pair in (kv for kv in self.default_arguments.items() if kv[0] not in sys.argv) for a in
                       pair)]
        self.args = getResolvedOptions(self.argv, ['JOB_NAME', 'DB_CONNECTION', 'DATALAKE_REGION'])
        self.job.init(self.args["JOB_NAME"], self.args)
        # Class 내 필요한 변수 초기화 
        self.db_compliance = 'de-db-replica-admin-connection'
        self.today_type1 = datetime.today().strftime('%Y%m%d')
        self.today_type2 = datetime.today().strftime('%Y_%m_%d')
        self.today = datetime.today().strftime("%Y-%m-%dT%H:%M:%S.%f")
        self.jurisdiction = 'de'
        self.target_bucket = 'compliance-develop'
        self.de_miss_data_file = f'-player-not-in-{self.jurisdiction}-{self.today_type2}.csv'  # db_compliance DB에 존재하지 않는  Data
        self._miss_data_file = f"-missing-data-{self.jurisdiction}-{self.today_type2}.csv"  #  DB에 존재하지 않는 Compliance Data
        self.data_diff_file = f'-{self.jurisdiction}-sync-{self.today_type2}.csv'  # 양 측의 Data가 정합성이 맞지 않는 Data
        self._bucket = f"/player_snapshot/{self.jurisdiction}/{self.today_type1}/"  #  Data File이 존재하는 버킷 위치
        self.target_bucket_sub_path = f"/player_sync_result/{self.jurisdiction}"  # Data Diff의 결과 csv file이 생성되어야 하는 bucket 위치
        self.diff_column_list = ["GGPassID", "FirstName", "LastName", "PostalCode", "City", "State", "Country",
                                 "StreetName", "HouseNumber", "Birthday", "PlaceOfBirth", "KycStatus", "RegisteredAt",
                                 "PlayerAccountStatus", "PlayerAccountStatusCause", "PlayerSuspensionStatus",
                                 "PlayerSuspensionStatusCause"]
        self.slack_title = "Error - DE Diff Check Glue Job"
        self.glue_url = "https://{region}.console.aws.amazon.com/gluestudio/home?region={region}#/editor/job/de-player-daily-snapshot-prod/runs"
        self.glue_job_name = "de-daily-data-sync"
        self.email_address = "dd.data.exchange@nsuslab.com"
        self.email_title = "[Error] DE Diff Check Glue Job"
        self.is_test = 0
        self.target_slack_channel = "notice-compliance-monitoring"
        self.slack_retry_error_msg = f"<br>Slack Retry 최대 횟수인 3번을 넘었습니다.<br>DE Diff Check Glue Job({self.glue_job_name})이 수행되지 않았으니 확인 필요 합니다."
        if self.is_test:
            self.target_slack_channel = 'notice-dx-alarm-test'
            self.target_bucket_sub_path = 'test/' + self.target_bucket_sub_path
            self._bucket = 'test/' + self._bucket

    def _retry(self, func, **kwargs):
        """
        Slack으로 알람 전달하는 과정에서도 오류가 발생할 경우를 대비 하기 위해 만든 Retry 로직. 
        최대 3번의 Retry 시도 한다.(각 시도는 5초 간격) 최대 Retry 시도 후에도 알람이 전송되지 않을 경우, 최종적으로 Email로 오류를 전달 한다.
        위의 과정에서도 예기치 못한 오류가 발생할 경우 Email로 오류를 전달하도록 처리한다. 
            - Slack Error Message 전달 후 Response = 200을 받게 되면, 함수를 종료 하게 된다.
        :param func: Error가 발생한 Function -> self._send_error_slack_message function
        :param kwargs: Error Message와 Error가 발생한 Function Name을 전달 받는다.
        :return: None
        """
        try_cnt = 0
        for i in range(1, 4):
            try:
                res = func(**kwargs)
                if res["ResponseMetadata"]["HTTPStatusCode"] == 200:
                    return
                else:
                    try_cnt += 1
                if try_cnt == 3:
                    self._send_error_email(subject=self.email_title,
                                           message=f"{self.slack_retry_error_msg}\n- HTTPStatusCode-> {res['ResponseMetadata']['HTTPStatusCode']}")
                time.sleep(5)
            except Exception as e:
                self._send_error_email(subject=self.email_title,
                                       message=f"Slack Error Message 최초 전달시 예기치 못한 오류가 발생함. -> {e}")
                break

    def __exception_handler(func):
        """
        수행될 각 Function 마다, Error 발생시 Slack 알람을 전달하기 위해 만든 decorator.
        Function에서 Error 발생시 _retry 함수로 Error가 발생한 Function을 전달 하여 처리한다. 
        :param func: Handler를 통해 전달 받을 Function.
        :return: wrapper -> function
        """

        @functools.wraps(func)
        def wrapper(self, *args, **kwargs):
            try:
                return func(self, *args, **kwargs)
            except Exception as e:
                self._retry(self._send_error_slack_message, message=e, func_name=func.__name__)

        return wrapper

    def _send_error_slack_message(self, message, func_name):
        """
        Slack으로 Error가 발생한 내용을 알람으로 전달 한다.
        :param message: Str -> Slack Message Body
        :param func_name: Str -> Error가 발생한 Function의 Name.
        :return: response -> Slack Message Body
        """
        sendErrorMessage = {
            "version": "1.0",
            "source": "custom",
            "content": {
                "textType": "client-markdown",
                "title": self.slack_title,
                "description": f"1. *Error Function*: {func_name}\n2. *Error Message*: {message}\n",
                "nextSteps": [
                    f"AWS Glue UI Refer to <{self.glue_url}|*_{self.glue_job_name}_* Glue Runs History>",
                    "Glue Run History Check",
                    "Code 내에서 Error 발생한 Function 확인이 필요 합니다."
                ],
            }
        }
        sns_client = boto3.client('sns')
        response = sns_client.publish(TopicArn=f"arn:aws:sns:{region}:{arnNumber}:{self.target_slack_channel}",
                                      Message=json.dumps(sendErrorMessage))
        return response

    def _send_error_email(self, subject, message):
        """
        Slack으로 Error Message를 전달하지 못할 경우 최종적으로 Email을 통해 Error를 알린다.
        :param subject: Str -> Email Title
        :param message: Str -> Email 본문 Error Message
        :return: None
        """
        ses_client = boto3.client(
            service_name='ses',
            region_name='us-east-1'
        )

        body_html = f"""
        <html>
            <head>
                <title>DE Diff Check Error Result</title>
                <style>
                    body {{
                        font-family: Arial, sans-serif;
                        background-color: #f4f4f4;
                        margin: 0;
                        padding: 20px;
                    }}
                    .container {{
                        background-color: #ffffff;
                        border-radius: 10px;
                        padding: 20px;
                        max-width: 600px;
                        margin: 0 auto;
                        box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
                    }}
                    h1 {{
                        color: #d9534f;
                        font-size: 24px;
                        text-align: center;
                    }}
                    p {{
                        font-size: 16px;
                        line-height: 1.6;
                        color: #333333;
                    }}
                    .error-details {{
                        background-color: #f9ecec;
                        border-left: 4px solid #d9534f;
                        padding: 10px;
                        margin: 20px 0;
                        color: #d9534f;
                    }}
                    .footer {{
                        font-size: 12px;
                        text-align: center;
                        color: #777777;
                        margin-top: 20px;
                    }}
                </style>
            </head>
            <body>
                <div class="container">
                    <h1>[Error] DE Diff Check via AWS Glue</h1>
                    <p>Dear User,</p>
                    <p>An error occurred during the DE Diff Check process. Please find the details below:</p>
                    <div class="error-details">
                        <strong>Error Details:</strong> <em>{message}</em>.
                    </div>
                    <p>Please address this issue at your earliest convenience.</p>
                    <p>Best regards,<br>DX Team</p>
                    <div class="footer">
                        <p>This is an automated message. Please do not reply.</p>
                    </div>
                </div>
            </body>
        </html>
        """
        ses_client.send_email(
            Source=self.email_address,
            Destination={
                "ToAddresses": [self.email_address]
            },
            Message={
                "Subject": {
                    'Charset': "UTF-8",
                    "Data": subject
                },
                "Body": {
                    "Text": {
                        'Charset': "UTF-8",
                        "Data": message
                    },
                    "Html": {
                        'Charset': "UTF-8",
                        "Data": body_html
                    }
                },
            }
        )

    @__exception_handler
    def _query_in_glue(self, query, connection):
        """
        Glue에서 Query를 수행하기 위함.
        :param query: SQL Query Statement
        :param connection: Glue Job이 사용할 Connections Name
        :return: awsglue.dynamicframe.DynamicFrame
        """
        result = self.glueContext.create_dynamic_frame.from_options(
            connection_type="custom.jdbc",
            connection_options={
                "query": query,
                "connectionName": connection,
            },
            transformation_ctx="result"
        )
        return result

    @__exception_handler
    def _get_de_data(self):
        """
        db_compliance DB에 Query를 수행하여 Player Data를 가져 온다.
            - 수정 일지
                - [DX-50] 
                    - Before
                        Query Statement내 `temporary_at` Column을 조건으로 작성. (where temporary_at is not null) 
                        - 이유: 에서 Lugas에 등록된 Player들만 Data Diff의 대상으로 선별하기 위함이다.
                    - After
                        Lugas에 등록되지 않은 Player들도 모두 Diff Check 대상에 포함하기 위해 temporary_at 조건절 제거. 
                - [DX-59]
                    - Before
                        Where 조건이 없었음.
                    - After
                        Where 조건에 날짜 값 추가 (이유: Glue Schedule이 동작하는 시간을 기준으로 조건을 넣어달라는 요청)
        :return: awsglue.dynamicframe.DynamicFrame
        """
        date_time = datetime.now(tz=timezone('UTC')).replace(hour=6, minute=0, second=0, microsecond=0).strftime(
            '%Y-%m-%d %H:%M:%S')
        query = f"""select *
                    from db_compliance.player
                    where registered_at <= '{date_time}'
                    """
        result = self._query_in_glue(query=query, connection=self.db_compliance)
        
        return result.toDF().select(
            col("ggpass_id").alias('GGPassID'),
            col('player_first_name').alias('FirstName'),
            col('player_last_name').alias( 'LastName'),
            col('post_code').alias('PostalCode'),
            col('place_of_residence').alias('City'),
            col('federal_state').alias('State'),
            col('country').alias('Country'),
            col('street').alias('StreetName'),
            col('house_number').alias('HouseNumber'),
            col('birth_date').cast('date').alias('Birthday'),
            col('place_of_birth').alias('PlaceOfBirth'),
            col('kyc_status').alias('KycStatus'),
            col('registered_at').alias('RegisteredAt'),
            col('account_status').alias('PlayerAccountStatus'),
            col('account_status_cause').alias('PlayerAccountStatusCause'),
            col('suspension_status').alias('PlayerSuspensionStatus'),
            col('suspension_reason').alias('PlayerSuspensionStatusCause'),
            col('temporary_at')
        ).fillna('NotRegistered', subset='PlayerAccountStatus')
        
        
    @__exception_handler
    def _get_s3_data(self):
        """
        s3에 존재하는  Player Data를 가져 와서 DynamicFrame으로 생성 한다.
        :return: awsglue.dynamicframe.DynamicFrame
        """
        s3_data = self.glueContext.create_dynamic_frame.from_options(
            format_options={
                "quoteChar": '"',
                "withHeader": True,
                "separator": ","
            },
            connection_type="s3",
            format="csv",
            connection_options={"paths": [f"s3://{self.target_bucket}/{self._bucket}"], "recurse": True}
        )
        return s3_data

    @__exception_handler
    def _leftanti_join_dataframe(self, df1, df2):
        """
        두 Dataframe을 leftanti Join을 통해, df2에 존재하지 않는 df1의 Data를 추출 한다. 
        :param df1: pyspark.sql.dataframe.DataFrame
        :param df2: pyspark.sql.dataframe.DataFrame
        :return: pyspark.sql.dataframe.DataFrame
        """
        result = df1.join(df2, on=["GGPassID"], how='leftanti')
        return result

    @__exception_handler
    def _write_down_s3(self, dataframe, file_name):
        """
        s3에 db_compliance에 존재하지 않는 Player Data를 csv File로 내려 적는다.
        :param dataframe: pyspark.sql.dataframe.DataFrame
        :param file_name: Str -> s3 Bucket File Full Path
        :return: None
        """
        dataframe.toPandas().to_csv(f"s3://{self.target_bucket}/{self.target_bucket_sub_path}/{file_name}", index=False)

    @__exception_handler
    def _de_data_transform(self, df):
        """
        DE DB에서 가져온 Data의 공백을 제거하기 위한 Transform 함수 추가 (2024.05.08).
        - 공백 제거 Column List
            1. place_of_residence -> .City
            2. federal_state -> .State
            3. country -> .Country
            4. street -> .StreetName
            5. house_number -> .HouseNumber
            6. place_of_birth -> .PlaceOfBirth
            7. kyc_status -> .KycStatus
            8. suspension_status -> .Suspension_Status
        :param df: Pyspark.DataFrame -> DE DB Data
        :return: Pyspark.DataFrame -> DE DB Data
        """
        trim_target_column_name = ['City', 'State', 'Country', 'StreetName', 'HouseNumber',
                                   'PlaceOfBirth']

        result = (df
              .select([trim(col(c)).alias(c) if c in trim_target_column_name else col(c) for c in df.columns])
              .fillna("Unknown", subset=["KycStatus"])
              .fillna("Active", subset=["PlayerSuspensionStatus"])
              )

        return result


    @__exception_handler
    def _check_diff_data(self, df1, df2):
        """
        Pyspark DataFrame 2개를 전달 받아 join을 통해 Data Difference가 존재할 경우 각각의 값을 반환 한다.
        - Lugas에 등록되지 않은 Player의 경우 Suspension_Status 컬럼의 값을 비교하지 않도록 한다.(2024.06.10 추가)
        :param df1:  pyspark.sql.dataframe.DataFrame
        :param df2: db_compliance pyspark.sql.dataframe.DataFrame
        :return: pyspark.sql.dataframe.DataFrame
        """
        from functools import reduce
        
        or_not_equal_columns = (list(set(self.diff_column_list) - set(["PlayerSuspensionStatus", "PlayerSuspensionStatusCause", "GGPassID"])))
        diff_data = (
            df1
            .join(df2, on=[df1["GGPassID"] == df2["GGPassID"]], how='inner')
            .where(
                reduce(lambda acc, c: acc | (df1[c] != df2[c]), or_not_equal_columns, 
                       (when(df2["temporary_at"].isNotNull(), df1["PlayerSuspensionStatus"] != df2["PlayerSuspensionStatus"]).otherwise(False))
                       )
            )
            .select(
                reduce(lambda acc, c: acc + [when(df1[c] != df2[c], df1[c]).alias(f"_{c}"), when(df1[c] != df2[c], df2[c]).alias(f"DE_{c}")], (list(set(self.diff_column_list) - set(["GGPassID"]))), [df1["GGPassID"]])
                )
        )
        return diff_data

    @__exception_handler
    def _send_slack_message(self, title, message, status):
        """
        Slack으로 Data Diff 결과를 전달 한다.
        :param title: Str -> Slack으로 전달될 Alarm Title
        :param message: Str -> Slack Message Body
        :param status: Str -> Slack Alarm 상태값 (ALARM: Error 상태를 의미, OK: 문제는 없지만 확인이 필요한 상태를 의미) 
        :return: None
        """
        complianceMessage = {
            "AlarmName": title,
            "AlarmDescription": "[DE] Core vs DE Data Sync Result",
            "AWSAccountId": "190490205267",
            "AlarmConfigurationUpdatedTimestamp": self.today[:-3] + "Z",
            "NewStateValue": status,
            "NewStateReason": message,
            "StateChangeTime": self.today[:-3] + "Z",
            "Region": "AP East (Hong Kong)",
            "AlarmArn": "arn:aws:cloudwatch:{region}:{arnNumber}:alarm:[DE] Core vs DE Data Sync Result",
            "Trigger": {
                "MetricName": "Invocations",
                "Namespace": "AWS/Glue",
                "StatisticType": "Statistic",
                "Statistic": "SUM",
                "Unit": "Count",
                "Dimensions": [
                    {
                        "value": self.glue_job_name,
                        "name": "GlueJobName"
                    }
                ],
                "Period": 60,
                "EvaluationPeriods": 1,
                "DatapointsToAlarm": 1,
                "ComparisonOperator": "GreaterThanThreshold",
                "Threshold": 0,
                "TreatMissingData": "missing",
                "EvaluateLowSampleCountPercentile": ""
            }
        }

        sns_client = boto3.client('sns')
        sns_client.publish(TopicArn=f"arn:aws:sns:{region}:{arnNumber}:{self.target_slack_channel}",
                           Message=json.dumps(complianceMessage))

    @__exception_handler
    def _make_slack_msg(self, noti_type, count, file_path):
        """
        Slack으로 전달 할 Message Body를 생성 한다.
        :param noti_type: 누락된 데이터가 어느 쪽인지를 판별하기 위한 기준 값
            - 1: db_compliance쪽에 누락된 데이터가 있는 경우
            - 2: 쪽에 누락된 데이터가 있는 경우
        :param count: Data Diff Count
        :param file_path: Diff Data가 담긴 s3 Bucket File Path
        :return: String -> Slack Message Body
        """
        if noti_type == 1:
            slack_msg = f"""{self.today_type2} {self.jurisdiction.upper()} Missing Player Count is {count}\nfile location : {file_path}\n"""
        else:
            slack_msg = f"""{self.today_type2}  Missing Player Count is {count}\nfile location : {file_path}"\n"""
        return slack_msg

    @__exception_handler
    def _make_data_diff_slack_msg(self, dataframe, count, file_path):
        """
        Data Diff가 존재할 경우 Slack으로 전달할 Message Body를 만든다.
        Data Diff를 비교한 개별 Column의 Sum값을 Dataframe의 Column으로 추가해 두었기 때문에, 해당 Column의 값을 가져와서 Slack Message에 추가 한다. 
        :param dataframe: pyspark.sql.dataframe.DataFrame
        :param count: int -> Data Diff Count
        :param file_path: str -> Data Diff 결과가 담긴 s3 File Full Path
        :return: Str -> Slack Message Body
        """
        first_row = dataframe.head(1)[0]
        
        result = {col: first_row[col] for col in dataframe.columns}
        slack_msg = f"""{self.today_type2} DE &  Data Diff Count is {count}\nfile location : {file_path}"\n"""
        
        for column in dataframe.columns:
            slack_msg += f"diff {column.replace('sum_', '')} : {result[column]}\n"
        return slack_msg

    @__exception_handler
    def execute(self):
        '''
        최종적으로 실행되어야 하는 function.
            1. s3에서 의 Data를 가져온다. 
            2. db_compliance DB에서 Query를 통해 Data를 가져온다.
            3. 의 Data를 1차 Transform 진행.
            4. DE DB의 Data의 공백 제거 및 Null 값 변환 처리를 위한 Transform 진행.
            5. db_compliance DB에 존재 하지 않는 Data가 있으면 s3에 csv File을 내려 적는다.
            6. 에 존재 하지 않는 Data가 있으면 s3에 csv File을 내려 적는다.
            7. , db_compliance DB 양 측의 Data를 비교하여 Data가 다른 Column의 Column별 Count수, Diff 부분을 s3 csv File로 내려 적는다.
            8. 에 데이터가 누락되어 있거나, db_compliance에 데이터가 누락되어있거나, Data Different가 존재할 경우 Slack으로 알람을 전달한다.
                - Slack Alarm Noti Case
                    1. 에 데이터 누락이 존재할 경우
                        - Status: OK
                        - Message:  Missing Player ...
                    2. db_compliance에 데이터 누락이 존재할 경우
                        - Status: ALARM
                        - Message: DE Missing Player ...
                    3. Data Different가 존재할 경우
                        - Status: OK
                        - Message: DE &  Data Diff ...
                    4. 1,2,3번 모두에 해당되지 않는 경우
                        - Status: OK
                        - Message: Data Consistency Check OK
        :return: None
        '''
        _dyf = self._get_s3_data()
        compliance_df = self._get_de_data()
        
        ##  Dynamicframe Transform
        new__df = _dyf.toDF().select(
            col("GGPassID"),
            col('FirstName'),
            col('LastName'),
            col('PostalCode'),
            trim(col('City')).alias('City'),
            trim(col('State')).alias('State'),
            trim(col('Country')).alias('Country'),
            trim(col('StreetName')).alias('StreetName'),
            trim(col('HouseNumber')).alias('HouseNumber'),
            col('Birthday').cast('date'),
            trim(col('PlaceOfBirth')).alias('PlaceOfBirth'),
            when(col('KycStatusCode') == 0, 'SignedUp')
            .when(col('KycStatusCode') == 1, 'EmailVerified')
            .when(col('KycStatusCode') == 2, 'DetailsFilled')
            .when(col('KycStatusCode') == 3, 'AgeVerified')
            .when(col('KycStatusCode') == 4, 'PoiVerified')
            .when(col('KycStatusCode') == 5, 'PoaVerified')
            .when(col('KycStatusCode') == 6, 'EddVerified').alias('KycStatus'),
            col('RegisteredAt'),
            col('PlayerAccountStatus'),
            col('PlayerAccountStatusCause'),
            col('PlayerSuspensionStatus'),
            col('PlayerSuspensionStatusCause'),
        ).cache()
        
        ## DE DataFrame Transform 
        new_de_df = self._de_data_transform(df=compliance_df).cache()
        ## 두 DataFrame의 Count수 확인
        df_count = [new__df.count(), new_de_df.count()]
        print("두 Dataframe의 갯수 확인: ", df_count)

        ## db_compliance에 존재하지 않는 Data를 csv file로 내려적는다.
        de_miss_player_df = self._leftanti_join_dataframe(df1=new__df, df2=new_de_df).cache()
        de_miss_count = de_miss_player_df.count()  # DE DB에 Miss Player Data Count
        self._write_down_s3(dataframe=de_miss_player_df, file_name=self.de_miss_data_file)  # Miss Player Data to s3
        print("DE DB에 존재하지 않는 Data Count: ", de_miss_count)

        ## 에 존재하지 않는 Data를 csv로 내려적는다.
        core_miss_player_df = self._leftanti_join_dataframe(df1=new_de_df, df2=new__df).cache()
        core_miss_count = core_miss_player_df.count()  # 에 존재하지 않는 Player Data Count
        self._write_down_s3(dataframe=core_miss_player_df, file_name=self._miss_data_file)  # Miss Player Data to s3
        print("에 존재하지 않는 Data Count: ", core_miss_count)

        ## 두 Dataframe에서 Data가 다른 경우, Diff Data들을 s3 File로 내려적기
        diff_df = self._check_diff_data(df1=new__df, df2=new_de_df).cache()
        diff_count = diff_df.count()
        self._write_down_s3(dataframe=diff_df, file_name=self.data_diff_file)
        print('Diff Data Count: ', diff_count)  # Diff Data Count Check            

        ## Slack으로 알람 전송
        if de_miss_count or core_miss_count or diff_count:
            total_msg = ''
            if de_miss_count > 0:
                # DE DB에 누락된 데이터가 존재할 경우 알람 전송
                de_message = self._make_slack_msg(noti_type=1,
                                                  count=de_miss_count,
                                                  file_path=f"s3://{self.target_bucket}/{self.target_bucket_sub_path}/{self.de_miss_data_file}")
                total_msg += de_message
            if core_miss_count > 0:
                # 에 누락된 데이터가 존재할 경우 알람 전송
                core_message = self._make_slack_msg(noti_type=2,
                                                    count=core_miss_count,
                                                    file_path=f"s3://{self.target_bucket}/{self.target_bucket_sub_path}/{self._miss_data_file}")
                total_msg += core_message
            if diff_count > 0:
                ##### 각 Column별 Data Diff Count를 구한다.
                aggregated_diff_counts = diff_df.select([spark_count(when(col(f"_{c}") != col(f"DE_{c}"), 1)).alias(f"sum_{c}") for c in list(set(self.diff_column_list) - set(["GGPassID", "temporary_at"]))])
                diff_message = self._make_data_diff_slack_msg(dataframe=aggregated_diff_counts,
                                                              count=diff_count,
                                                              file_path=f"s3://{self.target_bucket}/{self.target_bucket_sub_path}/{self.data_diff_file}")
                total_msg += diff_message
            self._send_slack_message(title="[DE] Core vs DE Data Sync Result",
                                     message=total_msg,
                                     status="ALARM" if de_miss_count > 0 else "OK")
        else:
            self._send_slack_message(title="[DE] Core vs DE Data Sync Result",
                                     message="Data Consistency Check OK",
                                     status="OK")

    def commit(self):
        """
        코드 파이프라인 배포에 의해서 자동으로 붙게됨. 
        임시 추가: 24.11.12
        :return: None
        """
        self.job.commit()


job = DataDiffCheck()
job.execute()